In [1]:
# Load the packages
import os
import pandas as pd
import numpy as np
import json
import ast
import time

In [2]:
# set our working directory
os.chdir("G:\projects\ForestPlanner_2015\Data\Work")
os.getcwd()

'G:\\projects\\ForestPlanner_2015\\Data\\Work'

In [3]:
FP_stands = pd.read_csv("FP_userstands_2015-10-09.csv", header = 0)
FP_stands.head()

,VARIANT,COND ID,AGE,TPA,STAND DETAILS
0,BM,5013,8,400,"{""classes"": [[""Ponderosa pine"", 0, 2, 400]], ""..."
1,BM,5013,8,175,"{""classes"": [[""Ponderosa pine"", 0, 2, 75], [""I..."
2,BM,6565,75,375,"{""classes"": [[""Ponderosa pine"", 12, 24, 125], ..."
3,BM,6616,125,10,"{""classes"": [[""Ponderosa pine"", 24, 36, 10]], ..."
4,BM,6678,30,75,"{""property"": ""trees_forestproperty_431"", ""clas..."


In [4]:
# Create a unique ID for each stand
FP_stands["User_StandID"] = FP_stands.index
FP_stands.head()

,VARIANT,COND ID,AGE,TPA,STAND DETAILS,User_StandID
0,BM,5013,8,400,"{""classes"": [[""Ponderosa pine"", 0, 2, 400]], ""...",0
1,BM,5013,8,175,"{""classes"": [[""Ponderosa pine"", 0, 2, 75], [""I...",1
2,BM,6565,75,375,"{""classes"": [[""Ponderosa pine"", 12, 24, 125], ...",2
3,BM,6616,125,10,"{""classes"": [[""Ponderosa pine"", 24, 36, 10]], ...",3
4,BM,6678,30,75,"{""property"": ""trees_forestproperty_431"", ""clas...",4


In [5]:
FP_stands["STAND DETAILS"] =  FP_stands["STAND DETAILS"].map(lambda d : ast.literal_eval(d))
FP_stands.head()

,VARIANT,COND ID,AGE,TPA,STAND DETAILS,User_StandID
0,BM,5013,8,400,"{u'property': u'trees_forestproperty_266', u'c...",0
1,BM,5013,8,175,"{u'property': u'trees_forestproperty_266', u'c...",1
2,BM,6565,75,375,"{u'property': u'trees_forestproperty_337', u'c...",2
3,BM,6616,125,10,"{u'property': u'trees_forestproperty_668', u'c...",3
4,BM,6678,30,75,"{u'classes': [[u'Ponderosa pine', 6, 12, 30], ...",4


In [6]:
FP_stands_new = FP_stands.join(pd.DataFrame(FP_stands["STAND DETAILS"].to_dict()).T)
FP_stands_new.head()

,VARIANT,COND ID,AGE,TPA,STAND DETAILS,User_StandID,classes,property
0,BM,5013,8,400,"{u'property': u'trees_forestproperty_266', u'c...",0,"[[Ponderosa pine, 0, 2, 400]]",trees_forestproperty_266
1,BM,5013,8,175,"{u'property': u'trees_forestproperty_266', u'c...",1,"[[Ponderosa pine, 0, 2, 75], [Incense cedar, 0...",trees_forestproperty_266
2,BM,6565,75,375,"{u'property': u'trees_forestproperty_337', u'c...",2,"[[Ponderosa pine, 12, 24, 125], [Grand fir, 0,...",trees_forestproperty_337
3,BM,6616,125,10,"{u'property': u'trees_forestproperty_668', u'c...",3,"[[Ponderosa pine, 24, 36, 10]]",trees_forestproperty_668
4,BM,6678,30,75,"{u'classes': [[u'Ponderosa pine', 6, 12, 30], ...",4,"[[Ponderosa pine, 6, 12, 30], [White fir, 2, 6...",trees_forestproperty_431


In [7]:
# split rows of species-diameter-density dictionary onto their own rows
rows = []
_ = FP_stands_new.apply(lambda row: [rows.append([row['User_StandID'], row['VARIANT'], row['COND ID'], row['AGE'], row['TPA'], row['property'], foresttype]) 
                         for foresttype in row.classes], axis=1)
FP_stands = pd.DataFrame(rows, columns = ["User_StandID", "Variant", "CondID", "Age", "TPA", "PropertyID", "ForestTypeRow"]).set_index(['User_StandID'])
FP_stands.head()

,Variant,CondID,Age,TPA,PropertyID,ForestTypeRow
User_StandID,,,,,,
0,BM,5013,8,400,trees_forestproperty_266,"[Ponderosa pine, 0, 2, 400]"
1,BM,5013,8,175,trees_forestproperty_266,"[Ponderosa pine, 0, 2, 75]"
1,BM,5013,8,175,trees_forestproperty_266,"[Incense cedar, 0, 2, 50]"
1,BM,5013,8,175,trees_forestproperty_266,"[Grand fir, 0, 2, 50]"
2,BM,6565,75,375,trees_forestproperty_337,"[Ponderosa pine, 12, 24, 125]"


In [8]:
# Explode each species-diameter-density list to its own set of columns
FP_stands.reset_index(inplace = True)
FP_stands[['Species', 'minDBH', 'maxDBH', 'FT_TPA']] = FP_stands['ForestTypeRow'].apply(pd.Series)
FP_stands.set_index("User_StandID", inplace = True)
FP_stands.head()

,Variant,CondID,Age,TPA,PropertyID,ForestTypeRow,Species,minDBH,maxDBH,FT_TPA
User_StandID,,,,,,,,,,
0,BM,5013,8,400,trees_forestproperty_266,"[Ponderosa pine, 0, 2, 400]",Ponderosa pine,0,2,400
1,BM,5013,8,175,trees_forestproperty_266,"[Ponderosa pine, 0, 2, 75]",Ponderosa pine,0,2,75
1,BM,5013,8,175,trees_forestproperty_266,"[Incense cedar, 0, 2, 50]",Incense cedar,0,2,50
1,BM,5013,8,175,trees_forestproperty_266,"[Grand fir, 0, 2, 50]",Grand fir,0,2,50
2,BM,6565,75,375,trees_forestproperty_337,"[Ponderosa pine, 12, 24, 125]",Ponderosa pine,12,24,125


In [9]:
# Unique common names in Forest Planner User data
FP_species = pd.unique(FP_stands.Species).tolist()
print(FP_species)

['Ponderosa pine', 'Incense cedar', 'Grand fir', 'White fir', 'Western larch', 'Lodgepole pine', 'Douglas-fir', 'Pacific madrone', 'California black oak', 'Sugar pine', 'Oregon white oak', 'Bigleaf maple', 'Canyon live oak', 'Noble fir', 'California-laurel', 'Western white pine', 'Red alder', 'Cherry', 'Willow', 'Western hemlock', 'Western redcedar', 'Black cottonwood', 'Mountain hemlock', 'Sitka spruce', 'Pacific yew', 'Pacific silver fir', 'Alaska cedar', 'Pacific dogwood', 'Oregon ash', 'Subalpine fir', 'Engelmann spruce']


In [12]:
# Load in the FVS species crosswalk
with open('G:\projects\ForestPlanner_2015\Data\Processed\FVS_species_crosswalk_2015-11-13.json') as data_file:    
    FVS_spp_crosswalk = json.load(data_file)

In [13]:
variants = pd.unique(FP_stands.Variant).tolist()

# check to make sure all common names in the Forest Planner User data have values in the FVS crosswalk dictionary
for species in FP_species:
    for variant in variants:
        # see if that species has an entry in the species crosswalk dictionary
        try:
            FVS_spp_crosswalk[species][variant]
        # if it's not in the FVS_spp_crosswalk dictionary, tell me
        except KeyError:
            print species
            pass

In [ ]:
# What the entries for the correct version of these common names are in FVS_spp_crosswalk
bad2good_commonnames = {'Incense cedar': 'Incense-cedar', 'California-laurel': 'California laurel', 'Cherry': 'Plum/cherry spp.', 'Willow': 'Willow spp.'}

newdict = {}
for badcommon in bad2good_commonnames.keys():
    newdict[badcommon] = {variant:'' for variant in variants}
    for variant in variants:
        print(badcommon + " ->"),
        print(variant + " ->"),
        print(FVS_spp_crosswalk[bad2good_commonnames[badcommon]][variant]),
        print
        newdict[badcommon][variant] = FVS_spp_crosswalk[bad2good_commonnames[badcommon]][variant]

print(newdict)


In [ ]:
# add the new listings to the FVS_spp_crosswalk
FVS_spp_crosswalk.update(newdict)

In [ ]:
# dump the updated FVS_spp_crosswalk dictionary back to json
# dump the species_crosswalk to a json file
with open('G:\projects\ForestPlanner_2015\Data\Processed\FVS_species_crosswalk_' + time.strftime("%Y-%m-%d") + '.json', 'w') as dumpsite:
    json.dump(FVS_spp_crosswalk, dumpsite)